In [1]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

#text processing
import nltk #nltk oder spacey
import string
import re #regular expression

import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
from torch.utils.data import TensorDataset, Dataset, DataLoader, RandomSampler, SequentialSampler

from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


#model(hugging face)
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW, get_scheduler

2025-01-24 14:08:16.302145: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-24 14:08:16.341967: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-24 14:08:16.341995: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-24 14:08:16.342019: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-24 14:08:16.350331: I tensorflow/core/platform/cpu_feature_g

In [3]:
df = pd.read_excel('df_without_greetings_and_closings.xlsx', index_col=0)
df

,ReceiverID,ActionType,NegoOutcome,Content,length,WordCount_basic,Content_Length,Sentence_Count,Word_Count_nltk,NegoOutcomeLabel,Sentiment,SentimentCategory
SenderID,,,,,,,,,,,,
70,69,Offer,FinalAccept,dear mr mendel hope you are doing well as the ...,2529,429,2529,17,470,1,0.213699,Positive
69,70,Counteroffer,FinalAccept,dear mr novak thank you for reaching out to me...,2579,439,2579,21,483,1,0.165002,Positive
70,69,Counteroffer,FinalAccept,dear mr mendel thank your kind response – no n...,2336,409,2336,15,454,1,0.222533,Positive
69,70,Counteroffer,FinalAccept,dear mr novak thank you for your swift reply i...,1961,345,1961,13,381,1,0.160333,Positive
70,69,Counteroffer,FinalAccept,dear mr mendel thank you for your proposal and...,1917,343,1917,10,376,1,0.122500,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...
851,856,Counteroffer,FinalReject,dear chis meyer thank you for your latest offe...,1755,338,1755,21,358,0,0.285340,Positive
856,851,Counteroffer,FinalReject,dear alex kramerthank you very much for your l...,1460,263,1460,17,294,0,0.282807,Positive
851,856,Counteroffer,FinalReject,dear chris meyerthank you for your fast answer...,673,133,673,10,144,0,0.251786,Positive


In [4]:
df['Content'] = df['Content'].astype(str)

In [5]:
nego_message_data=df['Content']
df_nego_message_data = pd.DataFrame(nego_message_data)
df_nego_message_data

,Content
SenderID,
70,dear mr mendel hope you are doing well as the ...
69,dear mr novak thank you for reaching out to me...
70,dear mr mendel thank your kind response – no n...
69,dear mr novak thank you for your swift reply i...
70,dear mr mendel thank you for your proposal and...
...,...
851,dear chis meyer thank you for your latest offe...
856,dear alex kramerthank you very much for your l...
851,dear chris meyerthank you for your fast answer...


In [6]:
df_nego_message_data['Content']

SenderID
70     dear mr mendel hope you are doing well as the ...
69     dear mr novak thank you for reaching out to me...
70     dear mr mendel thank your kind response – no n...
69     dear mr novak thank you for your swift reply i...
70     dear mr mendel thank you for your proposal and...
                             ...                        
851    dear chis meyer thank you for your latest offe...
856    dear alex kramerthank you very much for your l...
851    dear chris meyerthank you for your fast answer...
856    dear alex krameri am still very interested in ...
851    dear chris meyer im so sorry but i can not  ac...
Name: Content, Length: 2332, dtype: object

In [7]:
sentences = df_nego_message_data.Content.values
sentences = ["[CLS]"+ sentence +" [SEP]" for sentence in sentences]

labels= df.NegoOutcomeLabel.values

In [8]:
sentences

['[CLS]dear mr mendel hope you are doing well as the management of both our companies have agreed on building a fruitful partnership to face the upcoming disruptions in our market best with the innovation of green spirit i would like to introduce myself as sasa novak reaching out to you to tighten the general terms hurm ag and svn tech have established in their first round of discussions so far please find attached a proposal for the open issues of our joint venture which i would like to explain to you as following  directors first we would like to address the unsolved the issue of the number of directors that are sent by hurm ag and svn tech since we both agreed on a decisionmaking committee consisting of  members with a simple majority power we strongly suggest dividing those seats  so that we svn tech would be able to deal with the local authorities and partners successfully by using our strong local network and relationships  engineers to develop a thorough and convenient product s

In [9]:
labels

array([1, 1, 1, ..., 0, 0, 0])

In [10]:
tokenizer=DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print(tokenized_texts[0])
print(tokenized_texts[1])

['[CLS]', 'dear', 'mr', 'men', '##del', 'hope', 'you', 'are', 'doing', 'well', 'as', 'the', 'management', 'of', 'both', 'our', 'companies', 'have', 'agreed', 'on', 'building', 'a', 'fruit', '##ful', 'partnership', 'to', 'face', 'the', 'upcoming', 'disruption', '##s', 'in', 'our', 'market', 'best', 'with', 'the', 'innovation', 'of', 'green', 'spirit', 'i', 'would', 'like', 'to', 'introduce', 'myself', 'as', 'sas', '##a', 'novak', 'reaching', 'out', 'to', 'you', 'to', 'tighten', 'the', 'general', 'terms', 'hu', '##rm', 'ag', 'and', 'sv', '##n', 'tech', 'have', 'established', 'in', 'their', 'first', 'round', 'of', 'discussions', 'so', 'far', 'please', 'find', 'attached', 'a', 'proposal', 'for', 'the', 'open', 'issues', 'of', 'our', 'joint', 'venture', 'which', 'i', 'would', 'like', 'to', 'explain', 'to', 'you', 'as', 'following', 'directors', 'first', 'we', 'would', 'like', 'to', 'address', 'the', 'un', '##sol', '##ved', 'the', 'issue', 'of', 'the', 'number', 'of', 'directors', 'that', 'a

In [11]:
MAX_LEN = 512

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [12]:
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')

In [13]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [14]:
train_inputs, validation_inputs, train_labels, validation_labels= train_test_split(input_ids, labels, random_state=42, test_size=0.3)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=42, test_size=0.3)

In [15]:
train_inputs

array([[ 101, 7592, 2720, ...,    0,    0,    0],
       [ 101, 6203, 2720, ...,    0,    0,    0],
       [ 101, 6203, 4074, ...,    0,    0,    0],
       ...,
       [ 101, 6203, 4074, ...,    0,    0,    0],
       [ 101, 6203, 4074, ...,    0,    0,    0],
       [ 101, 6203, 2720, ...,    0,    0,    0]])

In [16]:
validation_inputs

array([[  101,  7592,  2720, ...,  2003,  2085,  5026],
       [  101,  7592,  2720, ..., 12342,  2007,  2115],
       [  101,  6203,  2720, ...,  1996,  4613,  2144],
       ...,
       [  101,  6203,  3782, ...,     0,     0,     0],
       [  101,  6203,  2720, ...,     0,     0,     0],
       [  101,  6203,  3782, ...,     0,     0,     0]])

In [17]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [18]:
#je mehr desto langsamer -> batchsize immer wieder ein bisschen anpassen und ausprobieren
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler=RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler=SequentialSampler(validation_data)
validation_dataloader =DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [19]:
# Device Setup: Force GPU if available
import torch
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Force CPU usage
device = torch.device("cuda")
#print("Using CPU for all computations.")

In [20]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = 2) 
model.to(device)
model.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [21]:
optimizer = AdamW(model.parameters(), lr=5e-5)

In [22]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for batch_idx, batch in enumerate(data_loader):
            ### Prepare data
            input_ids, input_mask, labels = [t.to(device) for t in batch]  # Move to GPU
        
            outputs = model(input_ids, attention_mask=input_mask, labels=labels)
            loss, logits = outputs['loss'], outputs['logits']
        
            _, predicted_labels = torch.max(logits, 1)
            num_examples += labels.size(0)
            correct_pred += (predicted_labels == labels).sum()

    return correct_pred.float()/num_examples * 100

In [23]:
import time

start_time = time.time()

NUM_EPOCHS = 10

for epoch in range(NUM_EPOCHS):
    
    model.train()
    
    for batch_idx, batch in enumerate(train_dataloader):
        ### Prepare data
        input_ids, input_mask, labels = [t.to(device) for t in batch]  # Move to GPU
        
        ### Forward
        outputs = model(input_ids, attention_mask=input_mask, labels=labels)
        loss, logits = outputs['loss'], outputs['logits']
    
        ### Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        ### Logging
        if not batch_idx % 250:
            print (f'Epoch: {epoch+1:04d}/{NUM_EPOCHS:04d} | '
                   f'Batch {batch_idx:04d}/{len(train_dataloader):04d} | '
                   f'Loss: {loss:.4f}')
            
    model.eval()

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_dataloader, device):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, validation_dataloader, device):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')

Epoch: 0001/0010 | Batch 0000/0051 | Loss: 0.6925
training accuracy: 68.50%
valid accuracy: 69.43%
Time elapsed: 0.56 min
Epoch: 0002/0010 | Batch 0000/0051 | Loss: 0.6048
training accuracy: 68.57%
valid accuracy: 69.43%
Time elapsed: 1.12 min
Epoch: 0003/0010 | Batch 0000/0051 | Loss: 0.4736
training accuracy: 82.05%
valid accuracy: 73.86%
Time elapsed: 1.68 min
Epoch: 0004/0010 | Batch 0000/0051 | Loss: 0.4003
training accuracy: 92.52%
valid accuracy: 77.00%
Time elapsed: 2.24 min
Epoch: 0005/0010 | Batch 0000/0051 | Loss: 0.2106
training accuracy: 93.75%
valid accuracy: 75.86%
Time elapsed: 2.79 min
Epoch: 0006/0010 | Batch 0000/0051 | Loss: 0.1575
training accuracy: 93.93%
valid accuracy: 67.43%
Time elapsed: 3.35 min
Epoch: 0007/0010 | Batch 0000/0051 | Loss: 0.1623
training accuracy: 99.57%
valid accuracy: 77.00%
Time elapsed: 3.91 min
Epoch: 0008/0010 | Batch 0000/0051 | Loss: 0.0083
training accuracy: 96.38%
valid accuracy: 75.00%
Time elapsed: 4.47 min
Epoch: 0009/0010 | Batch

In [24]:
from sklearn.metrics import classification_report
from collections import Counter

# Nach dem Training:
model.eval()

all_predictions = []
all_labels = []

with torch.no_grad():
    for batch in validation_dataloader:
        ### Prepare data
        input_ids, input_mask, labels = [t.to(device) for t in batch]
        
        ### Forward
        outputs = model(input_ids, attention_mask=input_mask)
        logits = outputs.logits  # Logits enthalten die rohen Vorhersagen des Modells
        
        ### Predictions und Labels sammeln
        predictions = torch.argmax(logits, dim=1)  # Klasse mit höchster Wahrscheinlichkeit
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Count der Vorhersagen
prediction_counts = Counter(all_predictions)
print("\nPrediction Counts:")
for label, count in prediction_counts.items():
    print(f"Class {label}: {count} predictions")

# Classification Report
print("\nClassification Report:")
print(classification_report(all_labels, all_predictions, target_names=["Class 0", "Class 1"]))


Prediction Counts:
Class 1: 521 predictions
Class 0: 179 predictions

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.61      0.51      0.56       214
     Class 1       0.80      0.86      0.83       486

    accuracy                           0.75       700
   macro avg       0.71      0.69      0.69       700
weighted avg       0.74      0.75      0.75       700

